In [ ]:
!pip install -q datasets transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_checkpoint = "facebook/convnextv2-tiny-1k-224"
batch_size = 20

In [ ]:

from datasets import load_metric

metric = load_metric("accuracy")

<ipython-input-4-350fcae91df9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [ ]:

data = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/summer24/lichen classification/Train_DATA")

Resolving data files:   0%|          | 0/683 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 683
    })
})

In [ ]:
labels = data["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'fruticose'

In [ ]:
from transformers import AutoImageProcessor

image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint)
image_processor

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

ConvNextImageProcessor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConvNextImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    RandomAffine,
    RandomRotation,
    Resize,
    ToTensor,
)

# normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
# if "height" in image_processor.size:
#     size = (image_processor.size["height"], image_processor.size["width"])
#     crop_size = size
#     max_size = None
# elif "shortest_edge" in image_processor.size:
#     size = image_processor.size["shortest_edge"]
#     crop_size = (size, size)
#     max_size = image_processor.size.get("longest_edge")

# # # train_transforms = Compose(
# # #         [
# # #             RandomResizedCrop(crop_size),
# # #             RandomHorizontalFlip(),
# # #             ToTensor(),
# # #             normalize,
# # #         ]
# # #     )

# transforms_model = Compose(
#         [
#             Resize(size),
#             CenterCrop(crop_size),
#             ToTensor(),
#             normalize,
#         ]
#     )

# # # def preprocess_train(example_batch):
# # #     """Apply train_transforms across a batch."""
# # #     example_batch["pixel_values"] = [
# # #         train_transforms(image.convert("RGB")) for image in example_batch["image"]
# # #     ]
# # #     return example_batch

# def preprocess(example_batch):
#     """Apply val_transforms across a batch."""
#     example_batch["pixel_values"] = [transforms_model(image.convert("RGB")) for image in example_batch["image"]]
#     return example_batch

In [ ]:

splits = data["train"].train_test_split(test_size=0.2)
train = splits['train']
val_ds = splits['test']

In [ ]:
# transforms_aug=Compose([
#     transforms.RandomAffine(degrees=(-5, 5), translate=(0.33, 0.33), scale=(1.0, 1.5)),
#     transforms.RandomChoice([
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomVerticalFlip(p=0.5)])
# ])

In [ ]:
# augmented_dataset = train.map(
#     lambda x: {"image": transforms_aug(x["image"])},
#     batched=False  # Process batches for efficiency
# )

In [ ]:
# from datasets import concatenate_datasets

# train_ds = concatenate_datasets([augmented_dataset, train])

In [ ]:
transform_rotate =transforms.RandomRotation(degrees=(0, 360))
transform_Hflip = transforms.RandomHorizontalFlip(p=1)
transform_Vflip = transforms.RandomVerticalFlip(p=1)
transform_affine = transforms.RandomAffine(degrees=0, translate=(0.33, 0.33), scale=(1.0, 1.5))
transform_blur = transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0))
# transform_sharpen = transforms.RandomCrop(size=(224, 224))

In [ ]:
from torchvision.transforms import functional as F

def equalize_image(image):
    return F.equalize(image)

def apply_transform(example, transform):
    example['image'] = transform(example['image'])
    return example

train_blur = train.map(lambda x: apply_transform(x, transform_blur))
train_Hflip = train.map(lambda x: apply_transform(x, transform_Hflip))
train_Vflip = train.map(lambda x: apply_transform(x, transform_Vflip))
train_affine = train.map(lambda x: apply_transform(x, transform_affine))
train_rotate = train.map(lambda x: apply_transform(x, transform_rotate))
# train_equalize = train.map(lambda x: apply_transform(x, equalize_image))

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets

train_ds = concatenate_datasets([train_blur,train_Hflip,train_Vflip,train_affine,train_rotate,train])

In [ ]:
train_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 3276
})

In [ ]:
val_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 137
})

In [ ]:
train_ds = train_ds.shuffle(seed=42)
train_ds = train_ds.flatten_indices()

Flattening the indices:   0%|          | 0/3276 [00:00<?, ? examples/s]

In [ ]:
# from transformers import DeiTFeatureExtractor

# # Load the DeiT feature extractor
# feature_extractor = DeiTFeatureExtractor.from_pretrained('facebook/deit-base-distilled-patch16-224')

# # Preprocess the dataset
# def preprocess(examples):
#     inputs = feature_extractor(examples['image'], return_tensors="pt")
#     inputs['labels'] = examples['label']
#     return inputs

# # Apply the transform to the dataset
# # dataset = dataset.with_transform(transform)


In [ ]:
def preprocess_function(examples):
    # Apply the image processor to all images in the batch
    inputs = image_processor(examples['image'], return_tensors="pt")
    return inputs

# Use map to apply the preprocess_function to the dataset
Train = train_ds.map(preprocess_function, batched=True)

# To avoid storing images multiple times, remove the original 'image' column if not needed
Train = Train.remove_columns("image")

Map:   0%|          | 0/3276 [00:00<?, ? examples/s]

In [ ]:
Train

Dataset({
    features: ['label', 'pixel_values'],
    num_rows: 3276
})

In [ ]:
Val = val_ds.map(preprocess_function, batched=True)

# To avoid storing images multiple times, remove the original 'image' column if not needed
Val = Val.remove_columns("image")

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

In [ ]:
from transformers import ConvNextV2ForImageClassification

model=  ConvNextV2ForImageClassification.from_pretrained(model_checkpoint, num_labels=4, ignore_mismatched_sizes=True)

Some weights of ConvNextV2ForImageClassification were not initialized from the model checkpoint at facebook/convnextv2-tiny-1k-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# from transformers import ConvNeXTV2Config, ConvNextV2Model

# # Create and initialize the configuration
# config = ConvNeXTV2Config(
#     num_labels=4,
#     problem_type="single_label_classification"  # Change according to your task
# )

# # Initialize the model with the configuration
# model = ConvNextV2Model(config)



In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer



In [ ]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-eurosat",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
import numpy as np

# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(eval_pred.label_ids,predictions, average='weighted')
    acc =metric.compute(predictions=predictions, references=eval_pred.label_ids)
    return {
    'accuracy': acc['accuracy'],
    'f1': f1,
    'precision': precision,
    'recall': recall
    }


In [ ]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([torch.tensor(example["pixel_values"])for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=Train,
    eval_dataset=Val,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [ ]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.268800,0.292727,0.897810,0.897104,0.904470,0.897810


In [ ]:
metrics = trainer.evaluate()
# some nice to haves:
trainer.log_metrics("eval", metrics)